In [22]:
import requests

# website for stations: https://till.mabe.at/rbl/?line=508&station=62517

# URL = "http://www.wienerlinien.at/ogd_realtime/monitor?rbl=1748&activateTrafficInfo=stoerungkurz&activateTrafficInfo=stoerunglang&activateTrafficInfo=aufzugsinfo&sender=hiti"
URL = "http://www.wienerlinien.at/ogd_realtime/monitor?stopId=1748&stopId=1718&stopId=1692&stopId=1708&activateTrafficInfo=stoerungkurz&activateTrafficInfo=stoerunglang&sender=hiti"
r = requests.get(url = URL)
r.json()["data"]["monitors"]

[{'locationStop': {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [16.3375036, 48.1789813]},
   'properties': {'name': '60200103',
    'title': 'Aßmayergasse',
    'municipality': 'Wien',
    'municipalityId': 90001,
    'type': 'stop',
    'coordName': 'WGS84',
    'attributes': {'rbl': 1692}}},
  'lines': [{'name': '59A',
    'towards': 'Bhf. Meidling S U',
    'direction': 'H',
    'platform': '1',
    'richtungsId': '1',
    'barrierFree': True,
    'realtimeSupported': True,
    'trafficjam': False,
    'departures': {'departure': [{'departureTime': {'timePlanned': '2023-01-02T23:46:00.000+0100',
        'timeReal': '2023-01-02T23:46:37.000+0100',
        'countdown': 14}},
      {'departureTime': {'timePlanned': '2023-01-03T00:01:00.000+0100',
        'timeReal': '2023-01-03T00:01:00.000+0100',
        'countdown': 28}},
      {'departureTime': {'timePlanned': '2023-01-03T00:16:00.000+0100',
        'timeReal': '2023-01-03T00:16:00.000+0100',
        'countdow

In [23]:
import json
with open('data.json', 'w') as f:
    json.dump(r.json(), f)

In [79]:
import pandas as pd

monitors = r.json()["data"]["monitors"]


def get_direction(line):
    b_direction = False

    # H-Directions go outward
    if line["direction"] == "H":
        b_direction = True

    # except for WLB. there its the other way around
    if line["name"] == "WLB":
        b_direction = not b_direction

    # return stirng
    if b_direction is True:
        return "outwards"
    else:
        return "center"

def to_series(line):
    ret = {
        "name": [line["name"]],
        "countdown": [line["departures"]["departure"][0]["departureTime"]["countdown"]],
        "outwards": [get_direction(line)]
    }
    return pd.DataFrame.from_dict(ret)

df = pd.DataFrame(columns=["name", "countdown", "outwards"])
for monitor in r.json()["data"]["monitors"]:
    series = to_series(monitor["lines"][0])
    df = pd.concat([df, series])

df


    

,name,countdown,outwards
0,59A,14,outwards
0,62,0,outwards
0,62,17,center
0,WLB,13,center
0,WLB,10,outwards


In [11]:
monitor

{'locationStop': {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [16.337913, 48.1788033]},
  'properties': {'name': '60200103',
   'title': 'Aßmayergasse',
   'municipality': 'Wien',
   'municipalityId': 90001,
   'type': 'stop',
   'coordName': 'WGS84',
   'attributes': {'rbl': 1718}}},
 'lines': [{'name': 'WLB',
   'towards': 'Baden Josefspl.',
   'direction': 'R',
   'platform': '2',
   'richtungsId': '2',
   'barrierFree': False,
   'realtimeSupported': True,
   'trafficjam': False,
   'departures': {'departure': [{'departureTime': {'timePlanned': '2023-01-02T23:26:00.000+0100',
       'timeReal': '2023-01-02T23:28:16.000+0100',
       'countdown': 10}},
     {'departureTime': {'timePlanned': '2023-01-02T23:41:00.000+0100',
       'timeReal': '2023-01-02T23:41:00.000+0100',
       'countdown': 23}},
     {'departureTime': {'timePlanned': '2023-01-02T23:56:00.000+0100',
       'timeReal': '2023-01-02T23:56:00.000+0100',
       'countdown': 38},
      'vehicle': {'